Import d3js and its dependencies

In [ ]:
d3 = await import("https://cdn.jsdelivr.net/npm/d3@7/+esm");
d3a = await import("https://cdn.jsdelivr.net/npm/d3-array@3.2.4/+esm");
d3c = await import("https://cdn.jsdelivr.net/npm/d3-collection@1.0.7/+esm");
d3p = await import("https://cdn.jsdelivr.net/npm/d3-path@3.1.0/+esm");
d3sh = await import("https://cdn.jsdelivr.net/npm/d3-shape@3.2.0/+esm");
d3s = await import("https://cdn.jsdelivr.net/npm/d3-sankey@0.12.3/+esm");
d3export = await import("https://cdn.jsdelivr.net/npm/d3-save-svg@0.0.2/+esm")

Parameters and queries

In [ ]:
$WorkspaceID = '2cc447f2-d22b-4f8c-8786-f522710c1b25'
$dateFiler = 'ago(1d)'

In [ ]:
$appQuery = @"
AZFWApplicationRule | where TimeGenerated  >= $dateFiler
"@

$networkQuery = @"
AZFWNetworkRule | where TimeGenerated  >= $dateFiler
"@

$natQuery = @"
AZFWNatRule | where TimeGenerated  >= $dateFiler
"@


Function Get-WhoIs {
    [cmdletbinding()]
    [OutputType("WhoIsResult")]
    Param (
        [parameter(Position = 0,
            Mandatory,
            HelpMessage = "Enter an IPV4 address to lookup with WhoIs",
            ValueFromPipeline,
            ValueFromPipelineByPropertyName)]
        [ValidateNotNullOrEmpty()]
        [ValidatePattern("^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$")]
        [ValidateScript( {
            #verify each octet is valid to simplify the regex
                $test = ($_.split(".")).where({[int]$_ -gt 254})
                if ($test) {
                    Throw "$_ does not appear to be a valid IPv4 address"
                    $false
                }
                else {
                    $true
                }
            })]
        [string]$IPAddress
    )

    Begin {
        Write-Verbose "Starting $($MyInvocation.Mycommand)"
        $baseURL = 'http://whois.arin.net/rest'
        #default is XML anyway
        $header = @{"Accept" = "application/xml"}

    } #begin

    Process {
        Write-Verbose "Getting WhoIs information for $IPAddress"
        $url = "$baseUrl/ip/$ipaddress"
        Try {
            $r = Invoke-Restmethod $url -Headers $header -ErrorAction stop
            Write-verbose ($r.net | Out-String)
        }
        Catch {
            $errMsg = "Sorry. There was an error retrieving WhoIs information for $IPAddress. $($_.exception.message)"
            $host.ui.WriteErrorLine($errMsg)
        }

        if ($r.net) {
            Write-Verbose "Creating result"
            [pscustomobject]@{
                PSTypeName             = "WhoIsResult"
                IP                     = $ipaddress
                Name                   = $r.net.name
                RegisteredOrganization = $r.net.orgRef.name
                City                   = (Invoke-RestMethod $r.net.orgRef.'#text').org.city
                StartAddress           = $r.net.startAddress
                EndAddress             = $r.net.endAddress
                NetBlocks              = $r.net.netBlocks.netBlock | foreach-object {"$($_.startaddress)/$($_.cidrLength)"}
                Updated                = $r.net.updateDate -as [datetime]
            }
        } #If $r.net
    } #Process

    End {
        Write-Verbose "Ending $($MyInvocation.Mycommand)"
    } #end
}

> NOTE

Before you can run below command, you should login to Azure

Get Application-level data

In [ ]:
$appDataSet = Invoke-AzOperationalInsightsQuery -WorkspaceId $WorkspaceID -Query $appQuery -ErrorAction Stop | Select-Object -ExpandProperty Results

Get Network-level data

In [ ]:
$netDataSet = Invoke-AzOperationalInsightsQuery -WorkspaceId $WorkspaceID -Query $networkQuery -ErrorAction Stop | Select-Object -ExpandProperty Results

Get NAT-leven data

In [ ]:
$natDataSet = Invoke-AzOperationalInsightsQuery -WorkspaceId $WorkspaceID -Query $natQuery -ErrorAction Stop | Select-Object -ExpandProperty Results

Convert and filter data

In [ ]:
$sourceIpGroups = $appDataSet | Group-Object SourceIp

$appDataSetConverted = 
foreach ($group in $sourceIpGroups) {
    $source = $group.Name
    $targets = $group.Group | Group-Object Fqdn, Action

    foreach($target in $targets) {
        [PSCustomObject]@{
            source = $source;
            sourceName = $source
            target = ($target.Name -split ",")[0]
            action = $target.Group[0].Action
            value = $target.Count
        }
    }
}

$appDataSetConverted[0..3] | ft -AutoSize

In [ ]:
$sourceIpGroups = $netDataSet | Group-Object SourceIp

$networkDataSetConverted = 
foreach ($group in $sourceIpGroups) {
    $source = $group.Name
    $targets = $group.Group | Group-Object DestinationIp, Action

    foreach($target in $targets) {
        [PSCustomObject]@{
            source = $source;
            sourceName = $source
            target = ($target.Name -split ",")[0]
            action = (($target.Name -split ",")[1]).trim()
            value = $target.Count
        }
    }
}

$networkDataSetConverted[0..3] | ft -AutoSize

In [ ]:
$sourceIpGroups = $natDataSet | Group-Object SourceIp, DestinationIp, DestinationPort

$natDataSetConverted = [System.Collections.Generic.List[PSCustomObject]]::new()
$natSources = [System.Collections.Generic.List[PSCustomObject]]::new()

foreach ($group in $sourceIpGroups) {
    $source = $group.Group[0].SourceIp
    $destination = "$($group.Group[0].DestinationIp)`:$($group.Group[0].DestinationPort)"
    $externalCount = $group.Count

    $record = [PSCustomObject]@{
        sourceName = $source;
        target = $destination
        value = $externalCount
        action = 'Allow'
    } 
    
    $natDataSetConverted.Add($record)
    $natSources.Add($record)
    
    $targets = $group.Group | Group-Object TranslatedIp, TranslatedPort

    foreach($target in $targets) {
        $natDataSetConverted.Add(
            [PSCustomObject]@{
                sourceName = $destination;
                target = $target.Group[0].TranslatedIp
                value = $target.Count
                action = 'Allow'
            }
        )

    }
}

$natDataSetConverted[0..3] | ft -AutoSize

convert data a bit more to simplify passing it to the JS kernel.  You should only use one of below cells. It depends on what you are tryimg to visualize

In [ ]:
# $dataSet | Export-Csv -Path .\sankeyDataSet.csv
$c = $appDataSetConverted | ConvertTo-Csv
$r = $c -join "`n"

In [ ]:
# $dataSet | Export-Csv -Path .\sankeyDataSet.csv
$c = $networkDataSetConverted | ConvertTo-Csv
$r = $c -join "`n"

In [ ]:
# $natDataSetConverted | Export-Csv -Path .\sankeyDataSet.csv
$c = ($locations + $natDataSetConverted)| ConvertTo-Csv
$r = $c -join "`n"

Prepare canvas

In [ ]:
<svg id="d3_target" style="width:100%;"></svg>

Visualize traffic flows

In [ ]:
#!set --value @pwsh:r --name csvStr

let showDenyInRed = true; // Global variable to track toggle state

// Specify the dimensions of the chart.
const width = 2000;
const height = 2000;//4800;

var data = d3.csvParse(csvStr);

let svg; // Global variable for the SVG element

// Function to initialize the SVG container
function initSvg() {
        svg = d3.select("#d3_target")
        .attr("width", width)
        .attr("height", height)
        .attr("viewBox", [10, 10, width, height])
        .attr("style", "max-width: 100%; height: auto; font: 10px sans-serif;");
}

function exportSvg() {
    d3export.save(svg.node());
}

// Function to draw the Sankey diagram
function drawSankey() {
    // Clear the existing SVG content
    svg.selectAll("*").remove();

    let filteredLinks = data.map((d, i) => ({
            source: d["sourceName"],
            target: d["target"],
            value: d["value"],
            action: d["action"],
            id: i
        }));

    const nodes = Array.from(
        new Set(filteredLinks.flatMap((d) => [d.source, d.target])),
        (name, id) => ({ name, id, colorId: Math.floor(Math.random() * 10) + 1 })
    );

    filteredLinks.forEach((d) => {
        d.source = nodes.find((e) => e.name === d.source).id;
        d.target = nodes.find((e) => e.name === d.target).id;
    });

    let data_final = { nodes, links: filteredLinks }; // Corrected to { nodes, links: filteredLinks }

    const sankey = d3s.sankey()
        .nodeSort((a, b) => a.id - b.id)
        .nodeAlign(d3s.sankeyLeft)
        .nodeId((d) => d.id)
        .linkSort(null)
        .nodeWidth(20)
        .nodePadding(20)
        .extent([[1, 50], [width - 1, height - 5]]);

    const color = d3.scaleOrdinal(d3.schemeSet3);

    // Drawing nodes
    const rect = svg.append("g")
        .attr("stroke", "#000")
        .selectAll("rect")
        .data(sankey(data_final).nodes)
        .join("rect")
        .attr("x", d => d.x0)
        .attr("y", d => d.y0)
        .attr("height", d => d.y1 - d.y0 >= 3 ? d.y1 - d.y0 : 3)
        .attr("width", d => d.x1 - d.x0)
        .attr("fill", d => color(d.colorId));

    rect.append("title")
        //.text(d => `${d.name}\n${d.targetLinks.length > 0 ? d.targetLinks.map(o => o.source.name).join("\n") : ""}`);
        .text(d => {
            const uniqueNames = new Set();

            d.targetLinks.forEach(link => uniqueNames.add(link.source.name));

            return Array.from(uniqueNames).join("\n")
        })

    // Creating gradients for links
    const defs = svg.append("defs");
    sankey(data_final).links.forEach((link, i) => {
        const gradient = defs.append("linearGradient")
            .attr("id", "gradient" + i)
            .attr("gradientUnits", "userSpaceOnUse")
            .attr("x1", link.source.x1)
            .attr("x2", link.target.x0);

        gradient.append("stop")
            .attr("offset", "0%")
            .attr("stop-color", color(link.source.colorId));

        gradient.append("stop")
            .attr("offset", "100%")
            .attr("stop-color", color(link.target.colorId));
    });

    // Drawing links with gradient or red color based on 'showDenyInRed' and link's action
    svg.append("g")
        .attr("fill", "none")
        .attr("stroke-opacity", 0.5)
        .selectAll("path")
        .data(sankey(data_final).links)
        .join("path")
        .attr("d", d3s.sankeyLinkHorizontal())
        .attr("stroke", (d, i) => showDenyInRed && d.action === "Deny" ? "red" : `url(#gradient${i})`)
        .attr("stroke-width", d => Math.max(1, d.width))
        .append("title")
        .text(d => `${d.source.name} → ${d.target.name}`);

    //Drawing labels for the nodes
    svg.append("g")
        .selectAll("text")
        .data(sankey(data_final).nodes)
        .join("text")
        .attr("x", d => d.x0 < width / 2 ? d.x1 + 6 : d.x0 - 6)
        .attr("y", d => (d.y1 + d.y0) / 2)
        .attr("dy", "0.35em")
        .attr("text-anchor", d => d.x0 < width / 2 ? "start" : "end")
        .text(d => d.name);
}

initSvg(); // Initialize the SVG container
drawSankey(); // Draw the Sankey diagram for the first time
exportSvg();